In [1]:
import math
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import emoji
from ipywidgets import Layout, interact, IntSlider,Label, HBox
import ipywidgets as widgets
import os
import sys
import h5py
from pathlib import Path

#### Put in the inputs

In [2]:
inclination = IntSlider(min=0,max=180,step=1) 
inclination_box = HBox([Label('Inclination (Deg):'), inclination])

image_width = IntSlider(min=50,max=500,step=1)
image_width_box = HBox([Label('Image Width (Pixels):'), image_width])

image_height = IntSlider(min=50,max=500,step=1)
image_height_box = HBox([Label('Image Height (Pixels):'), image_height])

cam_x = IntSlider(min=15,max=20,step=1)
cam_x_box = HBox([Label('Camera Size (X):'), cam_x])

cam_y = IntSlider(min=15,max=20,step=1)
cam_y_box = HBox([Label('Camera Size (Y):'), cam_y])

Frequency = IntSlider(min=10,max=30,step=1)
Frequency_box = HBox([Label('Frequency (x $10^{10}$) Hz:'), Frequency])

display(inclination_box)
display(image_width_box)
display(image_height_box)
display(cam_x_box)
display(cam_y_box)
display(Frequency_box)

In [4]:
MBH=1.233e43
M_UNIT=2.739e25
R_LOW=1
R_HIGH=1

INCLINATION = inclination.value
IMG_WIDTH = image_width.value
IMG_HEIGHT = image_height.value
CAM_SIZE_X = cam_x.value
CAM_SIZE_Y = cam_y.value
FREQS_PER_DEC=1
FREQ_MIN = Frequency.value*1.e10
STEPSIZE = 0.05
MAX_LEVEL = 1
#Writing the model.in file 
f = open('model.in','r')
text = f.readlines()
f.close()
text[0] = 'MBH\t\t(g)\t\t%.15e\n'%(MBH)
text[1] = 'M_UNIT\t\t(g)\t\t%.15e\n'%(M_UNIT)
text[2] = 'Rhigh\t\t(-)\t\t%d\n'%(R_LOW)
text[3] = 'Rlow\t\t(-)\t\t%d\n'%(R_HIGH)
text[5] = 'INCLINATION\t(deg)\t%d\n'%(INCLINATION)
text[6] = 'IMG_WIDTH\t(pixels)\t%d\n'%(IMG_WIDTH)
text[7] = 'IMG_HEIGHT\t(pixels)\t%d\n'%(IMG_HEIGHT)
text[8] = 'CAM_SIZE_X\t(Rg)\t\t%.15e\n'%(CAM_SIZE_X)
text[9] = 'CAM_SIZE_Y\t(Rg)\t\t%.15e\n'%(CAM_SIZE_Y)
text[10] = 'FREQS_PER_DEC\t(-)\t\t%d\n'%(FREQS_PER_DEC)
text[11] = 'FREQ_MIN\t(Hz)\t\t%.15e\n'%(FREQ_MIN)
text[12] = 'STEPSIZE\t(-)\t\t%.15e\n'%(STEPSIZE)
text[13] = 'MAX_LEVEL\t(-)\t\t%d\n'%(MAX_LEVEL)
f = open('model.in','w')
f.writelines(text)
f.close()

In [5]:
%%bash
./RAPTOR model.in bhac_data/data0202.dat 10

Model parameters:
MBH 		= 1.233e+43 
M_UNIT 		= 2.739e+25 
R_LOW 	= 1 
R_HIGH 	= 1 
INCLINATION 	= 57 
Observer parameters:
IMG_WIDTH 	= 335 
IMG_HEIGHT 	= 330 
CAM_SIZE_X 	= 19 
CAM_SIZE_Y 	= 19 
FREQS_PER_DEC 	= 1 
FREQ_MIN 	= 1.9e+11 
STEPSIZE 	= 0.05 
nws 3
ng 4
ng 6
ng 6



UNITS
L,T,M: 9.15604e+14 30541.3 2.739e+25
rho,u,B: 3.56836e-20 32.0708 20.0752

READING BHAC AMR SIMULATION DATA FROM bhac_data/data0202.dat
Reading HEADER...
nleafs 1992
levmax 3
ndim 3
ndir 3
nw 12
neqpar+nspecialpar 13 
it 278925
t 2.030000e+03
block size 0 32
block size 1 8
block size 2 8
eqpar 0 1.66667
eqpar 1 0.00679932
eqpar 2 1
eqpar 3 0.9375
eqpar 4 0.001
eqpar 5 6
eqpar 6 12
eqpar 7 100
eqpar 8 3.33333e-08
eqpar 9 1e-05
eqpar 10 1
eqpar 11 0
eqpar 12 0
spin 0.9375
1 1992 

Allocation memory...

Reading BODY...

 DONE 
7.396830e-06
DONE!
Number of frequencies to compute: 1
freq = +1.900000000000000e+11
block 0 of total 1089
block 10 of total 1089
block 20 of total 1089
block 30 of total 1089
block 40 of total 1089
block 50 of total 1089
block 60 of total 1089
block 70 of total 1089
block 80 of total 1089
block 90 of total 1089
block 100 of total 1089
block 110 of total 1089
block 120 of total 1089
block 130 of total 1089
block 140 of total 1089
block 150 of total 1089
block

### Plot the Data

In [3]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)
lim=0.001
offset=0
def plot_data(folder,tstart,tend):
    plt.figure(figsize=(15,15),facecolor='w')
    fig, ax1 = plt.subplots(nrows=1,ncols=1,figsize=(15,15))

#    data_id = 'I3.425455e+15'
    data_I_id =  'I1.900000e+11'
    for file in range(tstart,tend):
        t=(file+offset)%180
        print(file,t)
        file_name = folder+'/img_data_%d.h5'%t
        print(file_name)
        images = h5py.File(file_name,'r')
        print(images.keys())
        max_I=-100
        min_I=100
        for i in range(0,len(images[data_I_id])):
            current_I=np.max(images[data_I_id][i])
            if(max_I<current_I):
                max_I=current_I
            current_I=np.min(images[data_I_id][i])
            if(min_I>current_I):
                min_I=current_I
        print(len(images[data_I_id]))
        print(i)
        for i in range(0,len(images[data_I_id])):
            pixels=int(np.sqrt(len(images[data_I_id][i])))
            array_I=((np.reshape(images[data_I_id][i],(pixels,pixels))))
            alpha=((np.reshape(images['alpha'][i],(pixels,pixels))))
            beta=((np.reshape(images['beta'][i],(pixels,pixels))))

            figure_I=ax1.pcolormesh(alpha,beta,(array_I/max_I)**0.5,vmin=0,vmax=1,cmap='afmhot',shading='auto')
        fig.colorbar(figure_I,ax=ax1)
        ax1.set_xlabel(r"x [r$_g$]")
        ax1.set_ylabel(r"y [r$_g$]")
        ax1.set_aspect('equal', 'box')
        plt.savefig("plot_%d.png"%file, transparent=False)
        #plt.show()
        #images.close()
        #plt.clf()
        #images.close()
folder="output"
print(folder)
plot_data('output',10,11)

output
10 10
output/img_data_10.h5
<KeysViewHDF5 ['I1.900000e+11', 'Q1.900000e+11', 'U1.900000e+11', 'V1.900000e+11', 'alpha', 'beta']>
1089
1088


/usr/local/lib64/python3.6/site-packages/matplotlib/font_manager.py:1333: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
